<a href="https://colab.research.google.com/github/fabiobento/dnn-course-2024-1/blob/main/00_course_folder/hp_tuning/03_visualize_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Adaptado de [Visualize the hyperparameter tuning process](]https://keras.io/guides/keras_tuner/visualize_tuning/) de Haifeng Jin em  [Keras Developer Guides](https://keras.io/guides/).

In [11]:
!pip install keras-tuner -q

## Introdução

O KerasTuner imprime os registros na tela, incluindo os valores dos hiperparâmetros em cada tentativa para que o usuário monitore o progresso.

No entanto, a leitura dos registros não é intuitiva o suficiente para perceber as influências dos hiperparâmetros nos resultados.

Portanto, fornecemos um método para visualizar os valores dos hiperparâmetros e os resultados de avaliação correspondentes com figuras interativas usando o TensorBaord.

[O TensorBoard](https://www.tensorflow.org/tensorboard) é uma ferramenta útil para visualizar os experimentos de aprendizado de máquina.
> Ele pode monitorar as perdas e as métricas durante o treinamento do modelo e visualizar as arquiteturas do modelo.

A execução do KerasTuner com o TensorBoard fornecerá recursos adicionais para visualizar os resultados do ajuste de hiperparâmetros usando o plug-in HParams.

Usaremos um exemplo simples de ajuste de um modelo para o conjunto de dados de classificação de imagens MNIST para mostrar como usar o KerasTuner com o TensorBoard.

A primeira etapa é fazer o download e formatar os dados.

In [12]:
import numpy as np
import keras_tuner
import tensorflow as tf
from tensorflow.keras import layers

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
# Normalize os valores de pixel para o intervalo de [0, 1].
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Adicione a dimensão do canal às imagens.
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
# Imprima as formas dos dados.
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(60000, 28, 28, 1)
(60000,)
(10000, 28, 28, 1)
(10000,)


Em seguida, escrevemos uma função `build_model` para criar o modelo com hiperparâmetros e retornar o modelo.

Os hiperparâmetros incluem:
* o tipo de modelo a ser usado (perceptron de várias camadas ou rede neural convolucional)
* o número de camadas
* o número de unidades
* filtros
* se deve ser usado o dropout.

In [13]:

def build_model(hp):
    inputs = tf.keras.Input(shape=(28, 28, 1))
    # O tipo de modelo pode ser MLP ou CNN.
    model_type = hp.Choice("model_type", ["mlp", "cnn"])
    x = inputs
    if model_type == "mlp":
        x = layers.Flatten()(x)
        # O número de camadas do MLP é um hiperparâmetro.
        for i in range(hp.Int("mlp_layers", 1, 3)):
            # O número de unidades de cada camada são
            #  hiperparâmetros diferentes com nomes diferentes.
            x = layers.Dense(
                units=hp.Int(f"units_{i}", 32, 128, step=32),
                activation="relu",
            )(x)
    else:
        # O número de camadas da CNN também é um hiperparâmetro.
        for i in range(hp.Int("cnn_layers", 1, 3)):
            x = layers.Conv2D(
                hp.Int(f"filters_{i}", 32, 128, step=32),
                kernel_size=(3, 3),
                activation="relu",
            )(x)
            x = layers.MaxPooling2D(pool_size=(2, 2))(x)
        x = layers.Flatten()(x)

    # Um hiperparâmetro para saber se deve usar a camada de abandono.
    if hp.Boolean("dropout"):
        x = layers.Dropout(0.5)(x)

    # A última camada contém 10 unidades, que é o mesmo número de classes.
    outputs = layers.Dense(units=10, activation="softmax")(x)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)

    # Compilar o modelo.
    model.compile(
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"],
        optimizer="adam",
    )
    return model


Podemos fazer um teste rápido dos modelos para verificar se a compilação foi bem-sucedida tanto para CNN e MLP.

In [14]:

# Inicialize os `HyperParameters` e defina os valores.
hp = keras_tuner.HyperParameters()
hp.values["model_type"] = "cnn"
# Crie o modelo usando os `HyperParameters`.
model = build_model(hp)
# Teste se o modelo funciona com nossos dados.
model(x_train[:100])
# Imprima um resumo do modelo.
model.summary()

# Faça o mesmo para o modelo MLP.
hp.values["model_type"] = "mlp"
model = build_model(hp)
model(x_train[:100])
model.summary()

Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 5408)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 10)             │        54,090 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 54,410 (212.54 KB)

 Trainable params: 54,410 (212.54 KB)

 Non-trainable params: 0 (0.00 B)

Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │        25,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │           330 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,450 (99.41 KB)

 Trainable params: 25,450 (99.41 KB)

 Non-trainable params: 0 (0.00 B)

Inicialize o sintonizador `RandomSearch` com 10 tentativas e usando a acurácia de validação como a métrica para selecionar modelos.

In [15]:
tuner = keras_tuner.RandomSearch(
    build_model,
    max_trials=10,
    # Não retome a pesquisa anterior no mesmo diretório.
    overwrite=True,
    objective="val_accuracy",
    # Defina um diretório para armazenar os resultados intermediários.
    directory="/tmp/tb",
)

Inicie a pesquisa chamando `tuner.search(...)`. Para usar o TensorBoard, precisamos passar uma instância `keras.callbacks.TensorBoard` para os _callbacks_.

In [16]:
tuner.search(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=2,
    # Use o retorno de chamada do TensorBoard.
    # Os registros serão gravados em “/tmp/tb_logs”.
    callbacks=[tf.keras.callbacks.TensorBoard("/tmp/tb_logs")],
)

Trial 10 Complete [00h 00m 04s]
val_accuracy: 0.9592499732971191

Best val_accuracy So Far: 0.9825833439826965
Total elapsed time: 00h 01m 06s


Se estiver executando no Colab, os dois comandos a seguir mostrarão o TensorBoard
dentro do Colab.

`%load_ext tensorboard`

`%tensorboard --logdir /tmp/tb_logs`

Você tem acesso a todos os recursos comuns do TensorBoard. Por exemplo, você pode exibir as curvas de perda e métricas e visualizar o gráfico computacional dos modelos em diferentes tentativas.

![Curvas de perda e métricas](https://i.imgur.com/ShulDtI.png)
![Gráficos computacionais](https://i.imgur.com/8sRiT1I.png)

Além desses recursos, também temos uma guia HParams, na qual há três exibições.  Na exibição de tabela, você pode visualizar as 10 tentativas diferentes em uma tabela com os diferentes valores de hiperparâmetro e métricas de avaliação.

![Visualização da tabela](https://i.imgur.com/OMcQdOw.png)

No lado esquerdo, você pode especificar os filtros para determinados hiperparâmetros. Por exemplo, você pode especificar a visualização apenas dos modelos MLP sem a camada de dropout e com 1 a 2 camadas densas.

![Exibição de tabela filtrada](https://i.imgur.com/yZpfaxN.png)

Além da exibição de tabela, ele também oferece duas outras exibições, a exibição de coordenadas paralelas e a exibição de matriz de gráfico de dispersão. Elas são apenas métodos de visualização diferentes para os mesmos dados. Você ainda pode usar o painel à esquerda para filtrar os resultados.


Na visualização de coordenadas paralelas, cada linha colorida é uma tentativa.
Os eixos são os hiperparâmetros e as métricas de avaliação.

![Visualização de coordenadas paralelas](https://i.imgur.com/PJ7HQUQ.png)

Na visualização da matriz do gráfico de dispersão, cada ponto é um teste. Os gráficos são projeções das tentativas em planos com diferentes hiperparâmetros e métricas como eixos.

![Visualização da matriz do gráfico de dispersão](https://i.imgur.com/zjPjh6o.png)